In [1]:
import os
import pandas as pd
import tensorflow as tf
import glob
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.layers import Dense, Activation, Flatten, Conv2D, Lambda, MaxPooling2D, Dropout, Input
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K
from tensorflow.keras import layers
import pandas as pd
import tensorflow.keras.backend as K
from glob import glob

In [2]:
# HYPERPARAMETERS
BUCKETS = 5
MAX_ANGLE = 450 # May need to confirm this
one_hot_length = int(900/BUCKETS)
BINS = np.arange(0, MAX_ANGLE - (MAX_ANGLE % BUCKETS) + 2*BUCKETS, BUCKETS)
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64 # 128, 64, 32, 8, 6
DATA_VALUE = 'X'
Y_VALUE_1 = 'speed'
Y_VALUE_2 = 'steering_angle'
timesteps = 100 # 64, 5
interval = 10
shift_amount = 10
number_outputs = int(timesteps/interval)
img_shape = (160, 320, 3) #(100, 100, 3) # (3, 160, 320), # 
predicting_frame_num = 3
error_margin = 1

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
def get_data_list(dataset_folder, size='large'):

    assert size in ['large', 'small', 'medium'], "size must be one of 'large', 'small', 'medium'"
    
    files = glob(os.path.join(dataset_folder, '*'))
    sorted_files = sorted(files, key = os.path.getsize)

    if size == 'small':
        return sorted_files[:4]
    if size == 'medium':
        return sorted_files[:8]
    return sorted_files

In [1093]:
def _parse_features(example_proto):

    features = {'X': tf.io.FixedLenFeature([], dtype=tf.string), 
                'steering_angle': tf.io.FixedLenFeature([], dtype=tf.float32), 
                'speed': tf.io.FixedLenFeature([], dtype=tf.float32),}
    parsed_features = tf.io.parse_single_example(example_proto, features)

    image = tf.io.decode_raw(parsed_features['X'], tf.uint8)
    image = tf.reshape(image, shape=(3, 160, 320))
    image = tf.transpose(image, [1, 2, 0])

    speed = tf.cast(parsed_features['speed'], tf.float32)
    angle = tf.cast(parsed_features['steering_angle'], tf.float32)

    return image, (angle, speed)

In [1094]:
dataset_files = get_data_list(dataset_folder=r'D:\DL-CV-ML Projects\AUTOV\data\datasetB')

# Data Loader

In [1256]:
block_length = 300
interval_stride = 20
window_size = int(block_length / interval_stride)
print('Window size:', window_size)
print('Will result in inputs of:', (BATCH_SIZE, window_size, 160, 320, 3))

dataset = tf.data.Dataset.from_tensor_slices(dataset_files)
dataset = dataset.interleave(tf.data.TFRecordDataset, 
                             cycle_length=len(dataset_files), 
                             block_length=block_length, 
                             num_parallel_calls=tf.data.AUTOTUNE,
                             deterministic=True) # Every 300 * len(dataset_files) we repeat. 3300-3599 will be the images that follow 0-299 
dataset = dataset.window(size=block_length, shift=block_length, stride=1, drop_remainder=True) # Make every 300 files their own dataset
dataset = dataset.map(map_func = lambda x: x.skip( tf.random.uniform((), maxval=interval_stride, dtype=tf.int64) ), num_parallel_calls=tf.data.AUTOTUNE) # Skip value cannot be greater than stride
dataset = dataset.flat_map(lambda x: x.window(size=window_size, shift=block_length, stride=interval_stride, drop_remainder=True))
dataset = dataset.flat_map(lambda x: x.map(_parse_features, num_parallel_calls=tf.data.AUTOTUNE).batch(window_size))
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

Window size: 15
Will result in inputs of: (64, 15, 160, 320, 3)


In [1253]:
vds = dataset.take(2)

In [1254]:
vds = list(vds)

## Mimic Dataloader to test if it works

In [1126]:
class ArtificialDataset(tf.data.Dataset):

    def _generator(num_samples):
        dataset = tf.data.Dataset.range(139)
        for i in dataset:
            yield(i,)

    def __new__(cls, num_samples):
        return tf.data.Dataset.from_generator(cls._generator,
                                              output_signature = tf.TensorSpec(shape = (1,), dtype = tf.int64),
                                              args=(num_samples,))

In [1229]:
dataset = tf.data.Dataset.range(3)
dataset = dataset.interleave(lambda x: ArtificialDataset(x), cycle_length=3, block_length=30, 
                             num_parallel_calls=tf.data.AUTOTUNE,
                             deterministic=True)

dataset = dataset.window(size=30, shift=30, stride=1, drop_remainder=True)
dataset = dataset.map(lambda d: d.skip( tf.random.uniform((), maxval=5, dtype=tf.int64) ))
dataset = dataset.flat_map(lambda d: d.window(size=6, shift=5, stride=5, drop_remainder=True)) # If size = shift = stride = 1 then it will be the same as doing nothing and getting (BS, 1, size) else (BS, window_size, size)
# dataset = dataset.flat_map(lambda x: x.batch(6))

In [1237]:
for _, x in enumerate(dataset):
    # print(x)
    # print(list(list(x)[0]))
    j = list(x.as_numpy_iterator())
    print(j, len(j))
    # print(len(list(x.as_numpy_iterator())))

[array([1], dtype=int64), array([6], dtype=int64), array([11], dtype=int64), array([16], dtype=int64), array([21], dtype=int64), array([26], dtype=int64)] 6
[array([1], dtype=int64), array([6], dtype=int64), array([11], dtype=int64), array([16], dtype=int64), array([21], dtype=int64), array([26], dtype=int64)] 6
[array([1], dtype=int64), array([6], dtype=int64), array([11], dtype=int64), array([16], dtype=int64), array([21], dtype=int64), array([26], dtype=int64)] 6
[array([30], dtype=int64), array([35], dtype=int64), array([40], dtype=int64), array([45], dtype=int64), array([50], dtype=int64), array([55], dtype=int64)] 6
[array([33], dtype=int64), array([38], dtype=int64), array([43], dtype=int64), array([48], dtype=int64), array([53], dtype=int64), array([58], dtype=int64)] 6
[array([31], dtype=int64), array([36], dtype=int64), array([41], dtype=int64), array([46], dtype=int64), array([51], dtype=int64), array([56], dtype=int64)] 6
[array([60], dtype=int64), array([65], dtype=int64),

# Model

In [ ]:
class GRUConv(layers.Layer):

    def __init__(self, **kwargs):

        super(GRUConv, self).__init__(**kwargs)
    
    def call(self, x):

        return x

# Xtra

In [ ]:
# block_length = 300
# interval_stride = 10
# window_size = int(block_length / interval_stride)
# shift = 5
# print(window_size)

# dataset = tf.data.Dataset.from_tensor_slices(dataset_files)
# dataset = dataset.interleave(tf.data.TFRecordDataset, cycle_length=len(dataset_files), block_length=block_length) # Every 300 * len(dataset_files) we repeat. 3300-3599 will be the images that follow 0-299 
# dataset = tf.data.Dataset.from_tensor_slices([dataset])
# dataset = dataset.flat_map(lambda d: d.skip(tf.random.uniform((), maxval=9, dtype=tf.int64)))
# dataset = dataset.map(_parse_features, num_parallel_calls=tf.data.AUTOTUNE)
# dataset = dataset.window(window_size, shift=shift, stride=interval_stride, drop_remainder=True)
# Batch
# Shuffle